In [1]:
TE = joinpath(homedir(), "Downloads", "GSEA.test", "")

if isdir(TE)

    rm(TE; recursive = true)

end

mkdir(TE)

println("Made ", TE, ".")

Made /Users/kwat/Downloads/GSEA.test/.


In [2]:
using Revise
using BenchmarkTools

---

In [ ]:
using GSEA

In [ ]:
da = joinpath("data", "")

dw = Kwat.path.get_download_directory()

gm = joinpath(da, "h.all.v7.1.symbols.gmt")

;

## select

In [ ]:
GSEA.select(Kwat.gmt.read(gm), 0, 32)

## 

In [ ]:
se = joinpath(da, "setting.json")

## run (single-sample GSEA)

In [ ]:
GSEA.run_si(se, gm, joinpath(da, "nmf_k9_w.tsv"), joinpath(dw, "single_sample_gsea.tsv"))

## run (pre-rank GSEA)

In [ ]:
ke_ar, se_fe_, fe_sc =
    GSEA.run_pr(se, gm, joinpath(da, "gene_score.tsv"), joinpath(dw, "pre_rank_gsea", ""))

;

In [ ]:
fe_sc

## run (standard GSEA)

In [ ]:
ke_ar, se_fe_, sa_va =
    GSEA.run_st(se, gm, joinpath(da, "sample_value.tsv"), joinpath(dw, "standard_gsea", ""))

;

In [ ]:
sa_va

---

In [ ]:
rm(TE; recursive = true)

println("Removed ", TE, ".")